In [153]:
import os  # Manipulate Operating System
import win32gui, win32ui, win32con, win32clipboard  # Interface with Windows
import pandas as pd  # DataFrames
import numpy as np  # Array Transformation, Math
import warnings  # Ignore Warnings
import pyautogui  # Control Mouse and Keyboard

# Custom WindowCapture Class
from windowcapture import WindowCapture

# Warnings to Ignore
warnings.filterwarnings("ignore", category=UserWarning)

# Set to False so mouse cursor can be controlled beyond main screen
# Used due to multiple monitor setup
pyautogui.FAILSAFE = False

# Initialize WindowCapture class with name of NetPlay window
wincap = WindowCapture(window_name="NetPlay")
screenshot = wincap.get_screenshot()


In [154]:
screenshot.shape

(582, 723, 3)

In [155]:
player_location = pyautogui.locate(
    needleImage=r"C:\Users\Cooper\cv-mk64\img\netplay\Player.png",
    haystackImage=screenshot,
)

player_center = pyautogui.center(player_location)

mapping_location = pyautogui.locate(
    needleImage=r"C:\Users\Cooper\cv-mk64\img\netplay\Mapping.png",
    haystackImage=screenshot,
)

mapping_center = pyautogui.center(mapping_location)

print(f"Player Center: {player_center}")
print(f"Mapping Center: {mapping_center}")

player_screen_center = wincap.get_screen_position(player_center)
mapping_screen_center = wincap.get_screen_position(mapping_center)

print(f"Player Center Relative to Window Location on Screen: {player_screen_center}")
print(f"Mapping Center Relative to Window Location on Screen: {mapping_screen_center}")


Player Center: Point(x=383, y=133)
Mapping Center: Point(x=555, y=132)
Player Center Relative to Window Location on Screen: (-1396, 422)
Mapping Center Relative to Window Location on Screen: (-1224, 421)


In [99]:
player_screen_center[0]

-1366

In [171]:
def create_player_frame(
    player_location: tuple[int, int],
    mapping_location: tuple[int, int],
    wincap: WindowCapture,
) -> pd.DataFrame:

    # Translate Player Location Tuple into X and Y variables
    player_x = player_location[0]
    player_y = player_location[1]

    # Translate Mapping Location Tuple into X and Y variables
    mapping_x = mapping_location[0]
    mapping_y = mapping_location[1]

    # Bring the window to the front with the Window Handle ID
    win32gui.SetForegroundWindow(wincap.hwnd)

    # Abstraction for pyautogui interaction
    # Helper Function to copy selected cell value and paste into variable
    def click_helper(x: int, y: int, y_move: int) -> None:
        pyautogui.click(x, y)
        pyautogui.move(0, y_move)
        pyautogui.click()
        pyautogui.hotkey("ctrl", "c")
        
    def copy_paste() -> str:
        win32clipboard.OpenClipboard()
        data = win32clipboard.GetClipboardData()
        win32clipboard.EmptyClipboard()
        win32clipboard.CloseClipboard()
        
        return data

    # Copy Player Data from NetPlay Window
    click_helper(x=player_x, y=player_y, y_move=25)
    player_one = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=50)
    player_two = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=75)
    player_three = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=100)
    player_four = copy_paste()

    # Copy Controller Mapping (Port) Data from NetPlay Window
    click_helper(x=mapping_x, y=mapping_y, y_move=25)
    mapping_one = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=50)
    mapping_two = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=75)
    mapping_three = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=100)
    mapping_four = copy_paste()

    # Insert Values into Pandas DataFrame
    df = pd.DataFrame(
        data={
            "Player": [player_one, player_two, player_three, player_four],
            "Port": [mapping_one, mapping_two, mapping_three, mapping_four],
        }
    )

    df = df.drop_duplicates(subset="Player", keep="first")

    return df


In [172]:
df = create_player_frame(
    player_location=player_screen_center,
    mapping_location=mapping_screen_center,
    wincap=wincap,
)

In [173]:
df

,Player,Port
0,COOPER,"GC1,2"
